In [1]:
import numpy as np                                     # Matlab like syntax for linear algebra and functions
import matplotlib.pyplot as plt                        # Plots and figures like you know them from Matlab
import seaborn as sns                                  # Make the plots nicer to look at
from iminuit import Minuit                             # The actual fitting tool, better than scipy's
import sys                                             # Modules to see files and folders in directories
from scipy import stats
import scipy.signal as sp
import os # Method 1 of looping over files
from pathlib import Path # Method 2
import math
import sympy

In [2]:
sys.path.append('External_Functions')
from ExternalFunctions import Chi2Regression, BinnedLH, UnbinnedLH
from ExternalFunctions import nice_string_output, add_text_to_ax    # Useful functions to print fit results on figure

In [3]:
def read_csv(filename): # File reading
    dat = np.genfromtxt(filename, delimiter=',', skip_header=13, names=True)
    time = dat['Time_s']
    voltage = dat['Channel_1_V']
    return time, voltage

def polynomial(x, a, b, c): # Polynomial fit
    return a*(1/2)*x**2 + b*x + c

def gravity(ac, angle, angle_adjust, dBall, dRail): # Function for g
    g = (ac/math.sin(math.radians(angle + angle_adjust) )*( 1.0 + (0.4)*( ((dBall/100)**2.0) / ((dBall/100)**2.0 - (dRail/100)**2.0))  ))
    return g
    
ac,angle,angle_adjust,dBall,dRail = sympy.symbols('ac angle angle_adjust dBall dRail')
    
g_diff_ac = gravity(ac, angle, angle_adjust, dBall, dRail).diff(ac)
g_diff_angle = gravity(ac, angle, angle_adjust, dBall, dRail).diff(angle)
g_diff_adjust = gravity(ac, angle, angle_adjust, dBall, dRail).diff(angle_adjust)
g_diff_ball = gravity(ac, angle, angle_adjust, dBall, dRail).diff(dBall)
g_diff_rail = gravity(ac, angle, angle_adjust, dBall, dRail).diff(dRail)

#g_diff_ac = Derivative(grav,a)
#g_diff_angle = Derivative(grav,an)
#g_diff_adjust = Derivative(grav,ana)
#g_diff_ball = Derivative(grav,db)
#g_diff_rail = Derivative(grav,dr)
    
gdac = lambdify(ac,g_diff_ac,'numpy')
gdangle = lambdify(angle,g_diff_angle,'numpy')
gdadjust = lambdify(angle_adjust,g_diff_adjust,'numpy')
gdball = lambdify(dBall,g_diff_ball,'numpy')
gdrail = lambdify(dRail,g_diff_rail,'numpy')

TypeError: can't convert expression to float

In [ ]:
def g_e(ac, angle, angle_adjust, dBall, dRail, ac_e, angle_e, angle_a_e, dBall_e, dRail_e): # Error propagated g
    
    g_e = np.sqrt( ((gdac(ac))**{2})*(ac_e)**{2} + ((gdangle(angle))**{2})*(angle_e)**{2} + ((gdadjust(angle_adjust))**{2})*(angle_a_e)**{2} +((gdball(dBall))**{2})*(dBall_e)**{2} + ((gdrail(dRail))**{2})*(dRail_e)**{2} )
    
    return g_e

def weighted_avg_and_std(values, weights):
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)
    return (average, math.sqrt(variance))

In [ ]:
Paths = 'Data/Incline/Small Ball/First position' # Specify data

In [ ]:
# Arrays of data directly measured
Angle_first = [14.3,14.3,14.3,14.1,14.3]
Angle_first_e = [0.5,0.5,0.2,0.5,0.5]

Angle_f, Angle_f_e = weighted_avg_and_std(Angle_first,Angle_first_e)

Angle_second = [13.1,13.15,13.1,13.2,13.0]
Angle_second_e = [0.3,0.5,0.2,0.3,0.2]

Angle_s, Angle_s_e = weighted_avg_and_std(Angle_second,Angle_second_e)

Small_ball = [1.2,1.2,1.2,1.2,1.2]
Small_ball_e = [0.01,0.04,0.01,0.01,0.02]

Smol_bol, Smol_bol_e = weighted_avg_and_std(Small_ball,Small_ball_e)

D_rail = [0.612,0.628,0.605,0.6,0.63]
D_rail_e = [0.04,0.05,0.03,0.05,0.05]

D_rail_avg, D_rail_e = weighted_avg_and_std(D_rail,D_rail_e)

Gate_1 = [22.6,22.6,22.6,22.55,22.58]
Gate_2 = [39.15,39.15,39.2,39.15,39.19]
Gate_3 = [56.75,56.8,56.85,56.75,56.81]
Gate_4 = [73.3,73.25,73.35,73.26,73.24]
Gate_5 = [90.9,90.85,91.1,90.9,90.88]

Gate_list = [Gate_1,Gate_2,Gate_3,Gate_4,Gate_5]

Gate_1_e = [0.3,0.1,0.3,0.1,0.1]
Gate_2_e = [0.3,0.1,0.3,0.1,0.1]
Gate_3_e = [0.3,0.05,0.3,0.1,0.1]
Gate_4_e = [0.3,0.05,0.3,0.1,0.1]
Gate_5_e = [0.3,0.1,0.3,0.1,0.1]

Gate_list_e = [Gate_1_e,Gate_2_e,Gate_3_e,Gate_4_e,Gate_5_e] # Convert cm to m 

In [ ]:
# Adjustment to angle from measured differences
Angles_delta = Angle_s - Angle_f # Maybe
Angles_delta_e = np.sqrt(Angle_f_e**2 + Angle_s_e**2)

In [ ]:
# Weighted means of gates
Gates = []
Gates_e = []

for gate,gate_e in zip(Gate_list,Gate_list_e):
    average, std = weighted_avg_and_std(gate,gate_e)
    Gates.append(average)
    Gates_e.append(std)
    
Gates = [i / 100 for i in Gates] # Conversion from cm to m
Gates_e = [i / 100 for i in Gates_e]

In [ ]:
chi2_values = []
accels = []
accels_e = []
gs = []
gs_e = []

In [ ]:
for filename in os.listdir(Paths): # Loops over every file in specified folder
    f = os.path.join(Paths, filename)
    if os.path.isfile(f):
        time, voltage = read_csv(f)
        peaks = sp.find_peaks(voltage, height = 4, distance = 100)
        time_0 = []
        for i in peaks[0]:
            time_0.append(time[i])
            
        accel_chi2 = Chi2Regression(polynomial,time_0,Gates,Gates_e)
        accel_chi2.errordef = 1.0
        MinuitAccel = Minuit(accel_chi2,a=0,b=0,c=0)
        MinuitAccel.migrad()
        Chi2Accel = MinuitAccel.fval
        chi2_values.append(Chi2Accel) # Save chi2 to empty list
        
        # Receive acceleration and other stuff
        fit_accel, fit_vel, fit_start = MinuitAccel.values[:]
        fit_accel_e, fit_vel_e, fit_start_e = MinuitAccel.errors[:]
        accels.append(fit_accel)
        accels_e.append(fit_accel_e)
        
        # Ensure correct parameters are inserted
        acc,acc_e = weighted_avg_and_std(accels,accels_e)
        g = gravity(ac = acc, angle = Angle_f, angle_adjust = Angles_delta, dBall = Smol_bol, dRail = D_rail_avg)
        gs.append(g)
        g_ee = g_e(ac = acc, angle = Angle_f, angle_adjust = Angles_delta, dBall = Smol_bol, dRail = D_rail_avg, ac_e = acc_e, angle_e = Angle_f_e, angle_a_e = Angles_delta_e, dBall_e = Smol_bol_e, dRail_e = D_rail_e)
        gs_e.append(g_ee)

In [ ]:
print(chi2_values)
print(accels)
print(accels_e)
print(gs)

In [ ]:
# Take weighted mean of all g
true_g, true_g_e = weighted_avg_and_std(gs,gs_e)
print(true_g)
print(true_g_e)